In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.SparkContext  
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@2bcd9a5a
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSe

In [2]:
val frame = spark.read.option("header", true).option("inferSchema",true).csv("picsart_2.csv")
val df = frame.drop("advertising_id", "timezone", "duration", "app", "v").withColumn("date", frame("timestamp").cast(DateType))
df.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: long (nullable = true)
 |-- version: string (nullable = true)
 |-- date: date (nullable = true)



frame: DataFrame = [device_id: string, country_code: string ... 12 more fields]
df: DataFrame = [device_id: string, country_code: string ... 8 more fields]

In [11]:
/**Retention object that takes the following arguments: dataframe, date column, id column, and an integer as the number 
of days. Has two methods: 1) creates a dataframe returning each day's number of users, the number of users that returned back 
after the number of days indicated in the arguments, and the retention rate, 2) ret_by_event method which takes extra
arguments: the filtering column name and the exact filtering value, to return the retention rate of only that event's users.
*/

class Retention (frame:DataFrame, col_date:String, col_id:String, days:Int) {
def day_retention():DataFrame= {
  var df_1 = frame.select(col_date, col_id)
  var df_2 = df_1.groupBy(col_date).agg(collect_set(col(col_id)).as("id_set"))
  var df_join = df_1.join(df_2, df_1(col_date)<=>df_2(col_date), "inner").drop(df_1(col_date)).
                groupBy(col(col_date), col("id_set")).agg(countDistinct(col(col_id)).as("count_id_set")).sort(col(col_date)) 
  var window_spec = Window.orderBy(col_date)
  var df_join_added = df_join.withColumn("id_set_days", lead("id_set", 1) over window_spec)
  var df_join_added_1 = df_join_added.withColumn("common_id_set", array_intersect(col("id_set"), col("id_set_days")))
  var df_final = df_join_added_1.withColumn("count_ret_ids", size(col("common_id_set"))).withColumn("ret_rate", 
                 round(col("count_ret_ids")*100/col("count_id_set"), 2)).drop("id_set", "id_set_days", "common_id_set")
  df_final
}
def ret_by_event(col_event:String, event_type:String): DataFrame={
  var df_1 = frame.filter(col(col_event)===event_type).select(col_date, col_id)
  var df_2 = df_1.groupBy(col_date).agg(collect_set(col(col_id)).as("id_set"))
  var df_join = df_1.join(df_2, df_1(col_date)<=>df_2(col_date), "inner").drop(df_1(col_date)).
   groupBy(col(col_date), col("id_set")).agg(countDistinct(col(col_id)).as("count_id_set")).sort(col(col_date)) 
  var window_spec = Window.orderBy(col_date)
  var df_join_added = df_join.withColumn("id_set_days", lead("id_set", 1) over window_spec)
  var df_join_added_1 = df_join_added.withColumn("common_id_set", array_intersect(col("id_set"), col("id_set_days")))
  var df_final = df_join_added_1.withColumn("count_ret_ids", size(col("common_id_set"))).withColumn("ret_rate", 
                 round(col("count_ret_ids")*100/col("count_id_set"), 2)).drop("id_set", "id_set_days", "common_id_set")
                        
  df_final
}
 
}
//1.Calculate how many users who used app in X day:  a) returned the next day  b) returned 7 day after
val ret_1 = new Retention(df, "date", "device_id", 1)  //b) input 7 instead of 1 in the arguments
ret_1.day_retention.show()
ret_1.day_retention.filter(col("date")=!="2018-02-18").agg(avg(col("ret_rate"))).show()
//2.which tool has highest return rate (edit_text_apply, effect_apply, edit_item_open, edit_sticker_apply)
ret_1.ret_by_event("event_type", "edit_text_apply").show()
ret_1.ret_by_event("event_type", "edit_text_apply").filter(col("date")=!="2018-02-17").agg(avg(col("ret_rate"))).show()

+----------+------------+-------------+--------+
|      date|count_id_set|count_ret_ids|ret_rate|
+----------+------------+-------------+--------+
|2018-02-02|          94|           37|   39.36|
|2018-02-03|          41|           23|    56.1|
|2018-02-04|          39|           25|    64.1|
|2018-02-05|          45|           24|   53.33|
|2018-02-06|          35|           22|   62.86|
|2018-02-07|          34|           20|   58.82|
|2018-02-08|          36|           15|   41.67|
|2018-02-09|          29|           16|   55.17|
|2018-02-10|          31|           17|   54.84|
|2018-02-11|          28|           19|   67.86|
|2018-02-12|          33|           23|    69.7|
|2018-02-13|          29|           22|   75.86|
|2018-02-14|          40|           19|    47.5|
|2018-02-15|          33|           17|   51.52|
|2018-02-16|          30|           10|   33.33|
|2018-02-17|          20|            2|    10.0|
|2018-02-18|           2|           -1|   -50.0|
+----------+--------

defined class Retention
ret_1: Retention = ammonite.$sess.cmd10$Helper$Retention@6e681b8f

In [9]:
/**functions to group_by and count, find average, min, max of a column, and the same for each group*/
def GroupCount(frame:DataFrame, groupCols:Array[String], countCols:Array[String]):DataFrame = {
    var groupCols1:Array[Column] = groupCols.map(x=>col(x))
    var countCols1:Array[Column] = countCols.map(c => countDistinct(col(c)).alias(c))
    var dfFiltered = frame.groupBy(groupCols.head, groupCols.tail : _*).agg(countCols1.head, countCols1.tail : _*)
    dfFiltered
    }
def find_avg(frame:DataFrame, avg_col:String):DataFrame = {
    var dfAvgCol = frame.agg(round(avg(col(avg_col)), 2))
    dfAvgCol
  }
def find_avg_group(frame:DataFrame, group_col:String, avg_col:String) = {
    var dfAvgColGroup = frame.groupBy(col(group_col)).agg(round(avg(col(avg_col)), 2))
    dfAvgColGroup
}
def find_max(frame:DataFrame, max_col:String):DataFrame = {
     var dfMaxCol = frame.agg(max(col(max_col)))
     dfMaxCol
  }
def find_max_group(frame:DataFrame, group_col:String, max_col:String) = {
    var dfMaxColGroup = frame.groupBy(col(group_col)).agg(max(col(max_col)))
    dfMaxColGroup
}
def find_min(frame:DataFrame, min_col:String):DataFrame = {
     var dfMinCol = frame.agg(min(col(min_col)))
     dfMinCol
  }
def find_min_group(frame:DataFrame, group_col:String, min_col:String) = {
    var dfMinColGroup = frame.groupBy(col(group_col)).agg(min(col(min_col)))
    dfMinColGroup
}

//3) a) how much users did photo_like daily, b) in which day is it highest, c) if there is seasonality
//val df_new = df.filter(col("event_type")==="photo_like").withColumn("week_day", date_format(col("timestamp"), "u")) 
//a)
//val df_1 = GroupCount(df_new, Array("week_day", "date"), Array("device_id")).filter(col("date")=!="2018-02-02").sort("date")
//df_1.show(50)
//find_avg_group(df_1, "week_day", "device_id").sort(col("week_day")).show()
//b) in Feb 2018 on average on Tuesday users were more active in liking photos than on any other day.
//c)on Saturdays and Sundays, on average, more users did "photo_like", especially on Sunday because our data range contains
//one Sunday less than any other weekday. And on Mondays definitely the average number of users doing "photo_like" 
//is apparently low. 

defined function GroupCount
defined function find_avg
defined function find_avg_group
defined function find_max
defined function find_max_group
defined function find_min
defined function find_min_group

In [10]:
//4) what is conversion rate that users who did effect_try then did effect apply
 def conversion_rate(frame:DataFrame, event_col:String, stage1:String, stage2:String, id_col:String, 
                     time_col:String):Unit = {
     val window_spec = Window.partitionBy(id_col).orderBy(time_col)
     val count_stage1 = GroupCount(frame.select(id_col, time_col, event_col).filter(col(event_col)=== stage1),
                        Array(event_col), Array(id_col))
     val df_event = frame.select(id_col, time_col, event_col).filter(col(event_col)=== stage1 || col(event_col)===stage2).
                    withColumn("lead", lead(col(event_col), 1) over window_spec).filter(col(event_col)===stage1).
                    withColumn("concat_col", concat(col(event_col),lit(" , "),col("lead")))
     val count_stage2 = GroupCount(df_event.filter(col("concat_col")===stage1+" , "+stage2), Array("concat_col"),Array(id_col))
     val count_union = count_stage1.union(count_stage2)
     val new_df = count_union.union(List(("conversion_rate", (count_stage2.select(id_col).
                take(1)(0)(0).toString.toDouble*100/count_stage1.select(id_col).take(1)(0)(0).toString.toInt).toString))
                                        .toDF)
     println(new_df.show())
 }
conversion_rate(df, "event_type", "effect_try", "effect_apply", "session_id", "timestamp")

+--------------------+-----------------+
|          event_type|       session_id|
+--------------------+-----------------+
|          effect_try|              298|
|effect_try , effe...|              243|
|     conversion_rate|81.54362416107382|
+--------------------+-----------------+

()


defined function conversion_rate

In [24]:
//5) find how many users did effect apply and what is descriptive statistics for it (outliers, avg, median)
val df_5 = df.filter(col("event_type")==="effect_apply").select("event_type", "date", "device_id")
val df_5_short = GroupCount(df_5, Array("date"), Array("device_id")).withColumnRenamed("device_id", "count_device_id")
//count, avg, median, outliers
var description = df_5_short.describe()
var quantiles = df_5_short.stat.approxQuantile("count_device_id", Array(0.25, 0.5, 0.75), 0.0)
var Q1 = quantiles(0)
var median = quantiles(1)
var Q3 = quantiles(2)
var IQR = Q3-Q1
var lower_outlier_range = Q1 - 1.5*IQR
var upper_outlier_range = Q3 + 1.5*IQR
var outliers = df_5_short.filter(col("count_device_id")<=lower_outlier_range || col("count_device_id")>= 
               upper_outlier_range).withColumnRenamed("date", "outlier_date").withColumnRenamed("count_device_id",
               "outliers")
println(description.show())
println("Q1: "+Q1+"\n"+"median: "+median+"\n"+"Q3: "+Q3)
println(outliers.show(30))
        


+-------+-----------------+
|summary|  count_device_id|
+-------+-----------------+
|  count|               17|
|   mean|8.411764705882353|
| stddev|4.169214907051023|
|    min|                1|
|    max|               20|
+-------+-----------------+

()
Q1: 7.0
median: 8.0
Q3: 9.0
+------------+--------+
|outlier_date|outliers|
+------------+--------+
|  2018-02-04|      14|
|  2018-02-02|      20|
|  2018-02-17|       4|
|  2018-02-18|       1|
+------------+--------+

()


df_5: DataFrame = [event_type: string, date: date ... 1 more field]
df_5_short: DataFrame = [date: date, count_device_id: bigint]
description: DataFrame = [summary: string, count_device_id: string]
quantiles: Array[Double] = Array(7.0, 8.0, 9.0)
Q1: Double = 7.0
median: Double = 8.0
Q3: Double = 9.0
IQR: Double = 2.0
lower_outlier_range: Double = 4.0
upper_outlier_range: Double = 12.0
outliers: DataFrame = [outlier_date: date, outliers: bigint]

In [168]:
//6)* what first 5 events users do in their session start (should be used window functions, 
//find about it in the shared link above) - 5 events in array and % of users did it
//No general function yet.
var window = Window.partitionBy("session_id").orderBy("timestamp")
df.
withColumn("ev_1", lead("event_type", 1) over window).
withColumn("ev_2", lead("event_type", 2) over window).
withColumn("ev_3", lead("event_type", 3) over window).
withColumn("ev_4", lead("event_type", 4) over window).
withColumn("events",concat(col("event_type"),lit(","),col("ev_1"),lit(","),col("ev_2"),lit(","),col("ev_3"),lit(","),
        col("ev_4"))).select("session_id","events").groupBy("events").agg(countDistinct("session_id").as("count_5s")).
        orderBy(col("count_5s").desc).withColumn("percent_users",round(col("count_5s")/df.agg(countDistinct("session_id")).
        take(1)(0)(0).toString.toInt*100,2)).drop("count_5s").show()

+--------------------+-------------+
|              events|percent_users|
+--------------------+-------------+
|                null|        100.0|
|interstitial_ad_r...|         9.41|
|banner_ad_request...|         7.47|
|interstitial_ad_r...|         7.42|
|banner_ad_request...|         7.42|
|app_load,intersti...|         7.42|
|explore_open,app_...|         7.21|
|native_ad_waterfa...|         6.41|
|native_ad_respons...|         6.41|
|interstitial_ad_r...|         5.15|
|native_ad_respons...|          5.1|
|app_open,explore_...|         5.02|
|native_ad_request...|         5.02|
|explore_open,inte...|         4.85|
|shop_response,sho...|         4.81|
|effect_try,effect...|         4.72|
|banner_ad_request...|         4.39|
|native_ad_respons...|         4.26|
|shop_request,shop...|         4.22|
|shop_response,sho...|         4.01|
+--------------------+-------------+
only showing top 20 rows



window: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@665c3850

In [27]:
//7)* What are the most important metrics when we want to describe one session of user in app (edited)
//-session duration 
  
def session_duration(dataframe:DataFrame, session_id:String):Double = {
    var session_id_col = dataframe.filter(col("session_id")===session_id)
    var time_col = dataframe("timestamp")
    var new_frame = dataframe.groupBy(col("session_id")).agg(min(col("timestamp")).as("session_start"),
                    max(time_col).as("session_end")).withColumn("duration_seconds", unix_timestamp
                    (col("session_end"))-unix_timestamp(col("session_start")))
    var session_duration = new_frame.take(1)(0)(0).toString.toDouble
    session_duration
}
//-conversion rate of "event_try"/"event_apply"  //conversion function
//-ranking of most used events
def most_popular_ten_events(dataframe:DataFrame, session_id:String):Unit = {
        var frame_1 = dataframe.filter(col("session_id")===session_id)
        var frame_2 = frame_1.groupBy("event_type").agg(countDistinct(col("session_id"))).sort(desc("session_id")).limit(10)
        println(frame_2.show(false))
    }


defined function session_duration
defined function most_popular_ten_events